In [1]:
!pip install accelerate

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd /content/gdrive/MyDrive/CalmNest


Mounted at /content/gdrive
/content/gdrive/MyDrive/CalmNest


In [3]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

import random

In [4]:
def make_text_readable(text):
    # Replace periods, commas, and semicolons with the same punctuation followed by a newline character
    text = text.replace(". ", ".\n")
    text = text.replace(", ", ",\n")
    text = text.replace("; ", ";\n")
    return text

In [5]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [6]:
data=torch.load("database.pth")

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Supabase/gte-small")
model = AutoModel.from_pretrained("Supabase/gte-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/66.8M [00:00<?, ?B/s]

In [8]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [9]:
def get_suggestions(query,database):
 embeds=database[1]
 answers=database[2]

 query=[query]
 query_dict = tokenizer(query, max_length=512, padding=True, truncation=True, return_tensors='pt')
 result = model(**query_dict)
 query_embed = average_pool(result.last_hidden_state, query_dict['attention_mask'])
 query_embed = F.normalize(query_embed, p=2, dim=1)

 scores = (query_embed @ embeds.T).flatten()
 index  = torch.argmax(scores)

 if scores[index] > 0.90:
    return random.choice(answers[index])
 else:
    return "No Suggestions"

In [37]:
def infer(question,suggestion,pipe):
 messages = [
    {
        "role": "system",
        "content": "You are a friendly mental-health chatbot.",
    },
    {"role": "user", "content":f'''Give a short answer to this query using the provided suggestion (if any): \n Query : {question} \n Suggestions : {suggestion}'''},
 ]

 prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
 outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
 return outputs[0]["generated_text"].split("<|assistant|>")[1]

In [11]:
def get_response(query,data,pipe):
  sug=get_suggestions(query,data)
  return infer(query,sug,pipe)

In [38]:
q="But WHY IS IT ALWAYS MEE, why do I always have bad memories!"
print(get_response(q,data,pipe))


Sure, here's a short answer that answers the query:

Because you've had a lot of bad memories, it's understandable that you always have negative experiences in your mind. Perhaps you've experienced trauma, stress, or other negative events in the past, and these memories have been woven into your psyche. However, with time, your mind can begin to shift and heal, and you can develop a more positive outlook on life. Remember, everyone has bad memories, but it's how we respond to them that determines our mental health. With consistent effort and care, you can learn to let go of negative thoughts and emotions and cultivate a more positive outlook.
